In [27]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [47]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201))
my_dpi = 192

for i in range(201):
    arr = phi[:,:,i]

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as PNG & JPG of varying quality and open JPG's again to convert to np array
    plt.savefig('2.1.png', bbox_inches='tight', pad_inches=0)

    for q in [100, 75, 50, 25]:
        title = '2.1.' + str(q) + '.jpg'
        plt.savefig(title, bbox_inches='tight', pad_inches=0, dpi=(q*my_dpi/100))
        im = Image.open(title)
        output_array = np.asarray(im)
    
        # Loop through output_array and replace RGB by raw in copy
        copy = arr.copy()
        for row_index, row in enumerate(output_array):
            for col_index, col in enumerate(row):
                copy[row_index][col_index] = convertRGBToRaw(col[0])
    
        # Compute RMSE's and compression ratio and add to dataframe
        in_imsize = os.stat('2.1.png').st_size
        out_imsize = os.stat(title).st_size
        crcol = 'CR2.1.' + str(q)
        df.at[i, crcol] = in_imsize/out_imsize
        
        diff = copy - arr
        rmse = np.sqrt(np.mean(diff**2))
        rmsecol = 'RMSE2.1.' + str(q)
        df.at[i, rmsecol] = rmse
        
    plt.close()
# df.to_pickle("analysis2.1.pkl")
df

,CR2.1.100,RMSE2.1.100,CR2.1.75,RMSE2.1.75,CR2.1.50,RMSE2.1.50,CR2.1.25,RMSE2.1.25
0,4.148863,8.895564e-11,5.433290,2.413468e-09,7.250648,2.066075e-09,10.538945,7.935654e-10
1,4.128169,5.342171e-11,5.414409,1.362190e-09,7.321399,1.049199e-09,10.645278,4.895494e-10
2,3.985854,3.721427e-11,5.119674,1.065077e-09,6.837657,9.592430e-10,10.001224,3.481931e-10
3,3.891004,3.419594e-11,5.099813,6.668726e-10,6.733937,6.549022e-10,9.802158,4.132626e-10
4,3.999041,4.510328e-11,5.279747,9.854418e-10,6.860197,8.921002e-10,10.002398,5.118124e-10
...,...,...,...,...,...,...,...,...
196,3.425889,9.344500e-03,4.549869,1.076258e-01,5.946827,9.852367e-02,8.507975,3.752934e-02
197,3.333994,1.028376e-02,4.446794,1.147821e-01,5.795004,1.057475e-01,8.275523,4.032081e-02
198,3.298053,1.137544e-02,4.424648,1.285578e-01,5.714533,1.133334e-01,8.115479,4.539184e-02
199,3.361852,1.263808e-02,4.483221,1.408847e-01,5.828970,1.228165e-01,8.277571,5.007920e-02


In [33]:
# Load data and create dataframe
phi = np.load("Phi_201snap.npy")
df2 = pd.DataFrame(columns=['CR2.1', 'RMSE2.1'], index=np.arange(201))
my_dpi = 100

for i in range(201):
    arr = phi[:,:,i]

    # Plot image (raw automatically converted to grayscale)
    plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
    plt.imshow(arr, cmap='gray')
    plt.axis("off")

    # Save as PNG & JPG and open JPG again to convert to np array
    plt.savefig('analysis2.1.png', bbox_inches='tight', pad_inches=0)
    plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
    plt.close()
    im = Image.open('analysis2.1.jpg')
    output_array = np.asarray(im)

    # Create convert function to use to convert output_array back to raw values
    phimax, phimin = np.max(arr), np.min(arr)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin

    # Loop through output_array and replace RGB by raw in copy
    copy = arr.copy()
    for row_index, row in enumerate(output_array):
        for col_index, col in enumerate(row):
            copy[row_index][col_index] = convertRGBToRaw(col[0])

    # Compute RMSE's and compression ratio and add to dataframe
    in_imsize = os.stat('analysis2.1.png').st_size
    out_imsize = os.stat('analysis2.1.jpg').st_size
    df2.at[i, 'CR2.1'] = in_imsize/out_imsize
    
    diff = copy - arr
    rmse = np.sqrt(np.mean(diff**2))
    df2.at[i, 'RMSE2.1'] = rmse
# df.to_pickle("analysis2.1.pkl")
df2

,CR2.1,RMSE2.1
0,4.148863,0.0
1,4.128169,0.0
2,3.985854,0.0
3,3.891004,0.0
4,3.999041,0.0
...,...,...
196,3.425889,0.009345
197,3.333994,0.010284
198,3.298053,0.011375
199,3.361852,0.012638


In [5]:
# Computing single norms without for loop
i = 0

my_dpi = 192
phi = np.load("Phi_201snap.npy")
arr = phi[:,:,i] 
plt.figure(figsize=(294/my_dpi, 165/my_dpi), dpi=my_dpi)
plt.imshow(arr, cmap='gray')
plt.axis("off")
plt.savefig('analysis2.1.jpg', bbox_inches='tight', pad_inches=0)
plt.close()
im = Image.open('analysis2.1.jpg')
output_array = np.asarray(im)
output_array
copy = arr.copy()
phimax, phimin = np.max(arr), np.min(arr)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin
for i, row in enumerate(output_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])
norm = np.linalg.norm(copy - arr)
norm

1.3469364304465263e-08